# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/excertos_filtrados500.csv', sep = '|')
df.head()

,COD,DESCR_AREA,filtrado
0,1400,Responsabilidade,voto cuidar auto tomada conta especial instaur...
1,1700,Finanças Públicas,voto cuidar auto solicitação congresso naciona...
2,5700,Responsabilidade,relatório tratar embargo declaração opor exemp...
3,284,Direito Processual,voto relação outro processo judiciais tratar r...
4,298,Pessoal,voto relativo ato envolver senhor caber rememo...


In [2]:
df.shape

(13285, 3)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['DESCR_AREA']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13285, 10)

In [4]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 30000
limite_texto = 500
dim_vetor = 50

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado'].astype(str))

sequences = tokenizer.texts_to_sequences(df['filtrado'].astype(str))

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 22972 unique tokens.


In [5]:
max = 0
lens = []
for seq in sequences:
    lens.append(len(seq))
np.min(lens), np.mean(lens), np.max(lens), np.std(lens), len(lens), sum(pd.Series(lens) > limite_texto)

(12, 269.96507339104255, 509, 142.93907674539003, 13285, 19)

In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13285, 500)


# Treinamento

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM
from keras.optimizers import RMSprop
from keras.layers.core import Dropout


In [9]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1], trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 1,386,996
Trainable params: 1,386,996
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 168s 16ms/step - loss: 1.5567 - categorical_accuracy: 0.4770 - val_loss: 1.3399 - val_categorical_accuracy: 0.4912
Epoch 2/20
10628/10628 [==============================] - 166s 16ms/step - loss: 1.0310 - categorical_accuracy: 0.6612 - val_loss: 1.2553 - val_categorical_accuracy: 0.5499
E

In [10]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1], trainable=True))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
gru_3 (GRU)                  (None, 512)               864768    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 2,018,548
Trainable params: 2,018,548
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 162s 15ms/step - loss: 1.5797 - categorical_accuracy: 0.4902 - val_loss: 1.5483 - val_categorical_accuracy: 0.5265
Epoch 2/20
10628/10628 [==============================] - 161s 15ms/step - loss: 1.0384 - categorical_accuracy: 0.6645 - val_loss: 1.4672 - val_categorical_accuracy: 0.4957
E

In [11]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               314368    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 1,465,588
Trainable params: 1,465,588
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 202s 19ms/step - loss: 1.4899 - categorical_accuracy: 0.5079 - val_loss: 1.3187 - val_categorical_accuracy: 0.5634
Epoch 2/20
10628/10628 [==============================] - 201s 19ms/step - loss: 1.0760 - categorical_accuracy: 0.6584 - val_loss: 1.2411 - val_categorical_accuracy: 0.6056
E

In [12]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               1153024   
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 2,306,804
Trainable params: 2,306,804
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 203s 19ms/step - loss: 1.7461 - categorical_accuracy: 0.4116 - val_loss: 1.5639 - val_categorical_accuracy: 0.4351
Epoch 2/20
10628/10628 [==============================] - 201s 19ms/step - loss: 1.2262 - categorical_accuracy: 0.6098 - val_loss: 1.2541 - val_categorical_accuracy: 0.5939
E

In [8]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1124 10:51:57.063793 140268025624384 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1124 10:51:57.220042 140268025624384 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1124 10:51:57.225495 140268025624384 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1124 10:51:57.893259 140268025624384 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is d

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               471552    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 1,625,332
Trainable params: 1,625,332
Non-trainable params: 0
_________________________________________________________________


W1124 10:51:58.636789 140268025624384 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 342s 32ms/step - loss: 1.5190 - categorical_accuracy: 0.5061 - val_loss: 1.6748 - val_categorical_accuracy: 0.5374
Epoch 2/20
10628/10628 [==============================] - 296s 28ms/step - loss: 1.0752 - categorical_accuracy: 0.6486 - val_loss: 1.2990 - val_categorical_accuracy: 0.5905
Epoch 3/20
10628/10628 [==============================] - 291s 27ms/step - loss: 0.8879 - categorical_accuracy: 0.7187 - val_loss: 1.3390 - val_categorical_accuracy: 0.5469
Epoch 4/20
10628/10628 [==============================] - 291s 27ms/step - loss: 0.7896 - categorical_accuracy: 0.7494 - val_loss: 1.1245 - val_categorical_accuracy: 0.6707
Epoch 5/20
10628/10628 [==============================] - 290s 27ms/step - loss: 0.6781 - categorical_accuracy: 0.7833 - val_loss: 1.1105 - val_categorical_accuracy: 0.6327
Epoch 6/20
10628/10628 [==============================] - 290s 27ms/step - loss: 0.589

In [11]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               628736    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 1,782,516
Trainable params: 1,782,516
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 386s 36ms/step - loss: 1.5222 - categorical_accuracy: 0.5036 - val_loss: 1.4867 - val_categorical_accuracy: 0.5371
Epoch 2/20
10628/10628 [==============================] - 406s 38ms/step - loss: 1.1535 - categorical_accuracy: 0.6269 - val_loss: 1.4905 - val_categorical_accuracy: 0.4577
E

In [12]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               68736     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 1,218,676
Trainable params: 1,218,676
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 675s 64ms/step - loss: 1.5724 - categorical_accuracy: 0.4593 - val_loss: 1.4986 - val_categorical_accuracy: 0.5152
Epoch 2/20
10628/10628 [==============================] - 163s 15ms/step - loss: 1.0600 - categorical_accuracy: 0.6552 - val_loss: 1.1687 - val_categorical_accuracy: 0.6056
E

In [13]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
gru_3 (GRU)                  (None, 64)                22080     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 1,171,380
Trainable params: 1,171,380
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 158s 15ms/step - loss: 1.6084 - categorical_accuracy: 0.4329 - val_loss: 1.4991 - val_categorical_accuracy: 0.5088
Epoch 2/20
10628/10628 [==============================] - 157s 15ms/step - loss: 1.1265 - categorical_accuracy: 0.6375 - val_loss: 1.2340 - val_categorical_accuracy: 0.6172
E

In [16]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
gru_5 (GRU)                  (None, 512)               864768    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 2,080,372
Trainable params: 2,080,372
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 169s 16ms/step - loss: 1.

In [17]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
gru_6 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
Total params: 1,418,612
Trainable params: 1,418,612
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 159s 15ms/step - loss: 1.

In [18]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 50)           1148650   
_________________________________________________________________
gru_7 (GRU)                  (None, 512)               864768    
_________________________________________________________________
dense_10 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                650       
Total params: 2,046,900
Trainable params: 2,046,900
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 160s 15ms/step - loss: 1.

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 500, 50)           1148650   
_________________________________________________________________
gru_8 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_12 (Dense)             (None, 32)                8224      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                330       
Total params: 1,392,980
Trainable params: 1,392,980
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 161s 15ms/step - loss: 1.

In [21]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 500, 50)           1148650   
_________________________________________________________________
gru_10 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_15 (Dense)             (None, 32)                8224      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                330       
Total params: 1,392,980
Trainable params: 1,392,980
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 161s 15ms/step - loss: 1.

In [22]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 500, 50)           1148650   
_________________________________________________________________
gru_11 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_17 (Dense)             (None, 32)                8224      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                330       
Total params: 1,392,980
Trainable params: 1,392,980
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 159s 15ms/step - loss: 1.

In [23]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(GRU(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 500, 50)           1148650   
_________________________________________________________________
gru_12 (GRU)                 (None, 500, 256)          235776    
_________________________________________________________________
gru_13 (GRU)                 (None, 32)                27744     
_________________________________________________________________
dense_19 (Dense)             (None, 10)                330       
Total params: 1,412,500
Trainable params: 1,412,500
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 311s 29ms/step - loss: 1.7066 - categorical_accuracy: 0.4058 - val_loss: 1.6484 - val_categorical_accuracy: 0.5055
Epoch 2/20
10628/10628 [==================

In [24]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 500, 50)           1148650   
_________________________________________________________________
gru_14 (GRU)                 (None, 500, 512)          864768    
_________________________________________________________________
gru_15 (GRU)                 (None, 64)                110784    
_________________________________________________________________
dense_20 (Dense)             (None, 10)                650       
Total params: 2,124,852
Trainable params: 2,124,852
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 315s 30ms/step - loss: 1.5932 - categorical_accuracy: 0.4870 - val_loss: 1.4830 - val_categorical_accuracy: 0.4020
Epoch 2/20
10628/10628 [==================

In [25]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=64, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 500, 50)           1148650   
_________________________________________________________________
gru_16 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_21 (Dense)             (None, 10)                2570      
Total params: 1,386,996
Trainable params: 1,386,996
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 81s 8ms/step - loss: 1.6694 - categorical_accuracy: 0.4372 - val_loss: 1.4852 - val_categorical_accuracy: 0.5111
Epoch 2/20
10628/10628 [==============================] - 79s 7ms/step - loss: 1.1611 - categorical_accuracy: 0.6222 - val_loss: 1.3114 - val_categorical_accuracy: 0.4802
Epoch